In [1]:
from bs4 import BeautifulSoup

# Read the HTML file
with open('app/templates/elements/RightVerticalNavBar/Bowel/bowel_volume_control.html', 'r') as file:
    html_content = file.read()

In [2]:
soup = BeautifulSoup(html_content, 'html.parser')

In [3]:
button_tags = soup.find_all('button')

In [4]:
button_names = [button.get('name', '') for button in button_tags]

In [5]:
count = 0
for x in button_names:
    print("\"{}\"".format(x),end=",")
    count += 1
    if count % 4 == 0:
        print("\n")

"normal_bowel_sound_LUQ","normal_bowel_sound_LLQ","normal_bowel_sound_RUQ","normal_bowel_sound_RLQ",

"hyperactive_sound_LUQ","hyperactive_sound_LLQ","hyperactive_sound_RUQ","hyperactive_sound_RLQ",

"hypoactive_sound_LUQ","hypoactive_sound_LLQ","hypoactive_sound_RUQ","hypoactive_sound_RLQ",

"borborygmus_sound_LUQ","borborygmus_sound_LLQ","borborygmus_sound_RUQ","borborygmus_sound_RLQ",

"captement_sound_LUQ","captement_sound_LLQ","captement_sound_RUQ","captement_sound_RLQ",

"peritoneal_friction_rub_sound_LUQ","peritoneal_friction_rub_sound_LLQ","peritoneal_friction_rub_sound_RUQ","peritoneal_friction_rub_sound_RLQ",

"normal_bowel_sound_with_bruits_sound_LUQ","normal_bowel_sound_with_bruits_sound_LLQ","normal_bowel_sound_with_bruits_sound_RUQ","normal_bowel_sound_with_bruits_sound_RLQ",

"irritable_bowel_syndrome_sound_LUQ","irritable_bowel_syndrome_sound_LLQ","irritable_bowel_syndrome_sound_RUQ","irritable_bowel_syndrome_sound_RLQ",

"diarrhea_sound_LUQ","diarrhea_sound_LLQ","diarr

In [ ]:
normal_bowel_sound_with_bruits_buttons,constipation_buttons
bruits_due_to_renal_arteries_stenosis_buttons,paralytic_ileus_buttons
captement_buttons,hyperactive_buttons
normal_bowel_buttons,diarrhea_buttons
hypoactive_buttons,crohns_disease_buttons
irritable_bowel_syndrome_buttons,borborygmus_buttons
ulcerative_colitis_buttons,peritoneal_friction_rub_buttons